In [ ]:
import numpy             as np
import matplotlib.pyplot as plt
import tensorflow        as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip drive/MyDrive/Eye/OCT_20
#!unzip drive/MyDrive/Eye/file_downloaded

Streaming output truncated to the last 5000 lines.
  inflating: OCT_20/train/NORMAL/NORMAL-1045896-1.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1045896-2.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1045896-3.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1045896-4.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1047298-1.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1047298-2.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1047298-3.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1047298-4.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1047298-5.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1047298-6.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1047298-7.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1047298-8.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1047578-1.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1047578-2.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1047578-3.jpeg  
  inflating: OCT_20/train/NORMAL/NORMAL-1052624-1.jpeg  
  inflating: OCT_20/train/NORMAL/NORM

In [ ]:
batch_size = 1

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale            = 1./255
)

test_data = test_datagen.flow_from_directory(
    'OCT_20/test',
    target_size = (256,256),
    batch_size  = batch_size ,
    color_mode  = "grayscale",
    class_mode  = "categorical"
)

Found 1000 images belonging to 4 classes.


In [ ]:
# custom activation function
def custom_activation(output):
  logexpsum = tf.keras.backend.sum(tf.keras.backend.exp(output), axis=-1, keepdims=True)
  result    = logexpsum / (logexpsum + 1.0)

  return result

In [ ]:
#my code
#run this
##DME VS others
def define_discriminator2(in_shape=(256,256,1), n_classes=2):
  in_image = tf.keras.layers.Input(shape=in_shape)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(in_image)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)

  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  fe = tf.keras.layers.Dropout(0.1)(fe)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)

  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  #fe = tf.keras.layers.Dropout(0.1)(fe)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)

  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  fe = tf.keras.layers.Dropout(0.1)(fe)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)

  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  fe = tf.keras.layers.Flatten()(fe)
  #fe = tf.keras.layers.Dropout(0.1)(fe)
  fe = tf.keras.layers.Dense(n_classes)(fe)

  c_out_layer = tf.keras.layers.Activation("softmax")(fe)
  c_model     = tf.keras.models.Model(in_image, c_out_layer)

  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.001, 50, 0.96, staircase=True, name=None)
  c_model.compile(loss = "categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate =lr_schedule,  beta_1 = 0.5,) , metrics = ["accuracy"])

  #tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2)

  d_out_layer = tf.keras.layers.Lambda(custom_activation)(fe)
  d_model     = tf.keras.models.Model(in_image, d_out_layer)
  d_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule, beta_1 = 0.5))
  return d_model, c_model

In [ ]:
#my code
#run this
## 3 class discriminator
def define_discriminator3(in_shape=(256,256,1), n_classes=3):
  in_image = tf.keras.layers.Input(shape=in_shape)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(in_image)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  fe = tf.keras.layers.Dropout(0.1)(fe)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  fe = tf.keras.layers.Dropout(0.1)(fe)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  fe = tf.keras.layers.Dropout(0.1)(fe)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  fe = tf.keras.layers.Flatten()(fe)
  fe = tf.keras.layers.Dropout(0.1)(fe)
  fe = tf.keras.layers.Dense(n_classes)(fe)

  c_out_layer = tf.keras.layers.Activation("softmax")(fe)
  c_model     = tf.keras.models.Model(in_image, c_out_layer)
  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(0.001, 50, 0.96, staircase=True, name=None)
  c_model.compile(loss = "categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate =lr_schedule,  beta_1 = 0.5), metrics = ["accuracy"])

  d_out_layer = tf.keras.layers.Lambda(custom_activation)(fe)
  d_model     = tf.keras.models.Model(in_image, d_out_layer)
  d_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule , beta_1 = 0.5))
  return d_model, c_model

In [ ]:
d2_model, c2_model = define_discriminator2()
d3_model, c3_model = define_discriminator3()

In [ ]:
d3_model.load_weights('/content/drive/MyDrive/GAN_20p/OTHER_TO_3/saved_model/new_d_model_3class_97')
c3_model.load_weights('/content/drive/MyDrive/GAN_20p/OTHER_TO_3/saved_model/new_c_model_3class_97')


In [ ]:
c2_model.load_weights('/content/drive/MyDrive/GAN_20p/DME_VS_OTHER/saved_model/c_model_new_with_drop')
d2_model.load_weights('/content/drive/MyDrive/GAN_20p/DME_VS_OTHER/saved_model/d_model_new_with_drop')

In [ ]:
predicted = []
real = []
for index , (image,label) in enumerate(test_data):
  if index== 1000:
    break
  output = c2_model.predict(image)
  real.append(np.argmax(label))

  if np.argmax(output) == 0:
    predicted.append(1)
  elif np.argmax(output) == 1:
    output = c3_model.predict(image)
    if np.argmax(output) == 0:
        predicted.append(0)
    elif np.argmax(output) == 1:
        predicted.append(2)
    elif np.argmax(output) == 2:
        predicted.append(3)




1/1 [==============================] - 0s 26ms/step


In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(real, predicted))


0.932


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(predicted, real)

array([[237,  12,  11,   1],
       [ 10, 213,   0,   3],
       [  3,   1, 237,   1],
       [  0,  24,   2, 245]])

In [ ]:
predicted_categories = tf.argmax(y_pred, axis=1)
true=[]
predicted=[]
for index , (image,label) in enumerate(test_data):
  if index== 1000:
    break
  true.append(label)
  predicted.append(c_model.predict(image))

NameError: ignored

In [ ]:
predicted_categories = tf.argmax(predicted, axis=1)
true_cat= []
for item in true:
  true_cat.append(np.argmax(item))
predicted_cat= []
for item in predicted:
  predicted_cat.append(np.argmax(item))
confusion_matrix(predicted_cat, true_cat)

# Test Data
## CNV
## Drussen
## DME
## Normal

predicted = []
real = []
for image, label in test_data:
    output = c2_model(image)
    real.append(label)
    If output = 0:
      predicted.append(1)
    If output = 1:
      output = c3_model(image)
      if output == 0:
        predicted.append(0)
      elif output == 1:
        predicted.append(2)
      elif output == 2:
        predicted.append(3)

real , predicted compare

In [ ]:
#reserve real code
#NOT run this
## 3 class discriminator

# def define_discriminator3(in_shape=(256,256,1), n_classes=3):
#   in_image = tf.keras.layers.Input(shape=in_shape)

#   fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(in_image)
#   fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
#   fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

#   #fe = tf.keras.layers.Dropout(0.7)(fe)

#   fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
#   fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
#   fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

#   #fe = tf.keras.layers.Dropout(0.7)(fe)

#   fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
#   fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
#   fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

#   #fe = tf.keras.layers.Dropout(0.7)(fe)

#   fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
#   fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
#   fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

#   fe = tf.keras.layers.Flatten()(fe)
#   fe = tf.keras.layers.Dropout(0.4)(fe)
#   fe = tf.keras.layers.Dense(n_classes)(fe)

#   c3_out_layer = tf.keras.layers.Activation("softmax")(fe)
#   c3_model     = tf.keras.models.Model(in_image, c3_out_layer)
#   c3_model.compile(loss = "categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002,  beta_1 = 0.5), metrics = ["accuracy"])

#   d3_out_layer = tf.keras.layers.Lambda(custom_activation)(fe)
#   d3_model     = tf.keras.models.Model(in_image, d3_out_layer)
#   d3_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002, beta_1 = 0.5))
#   return d3_model, c3_model

In [ ]:
#reserve real code
#NOT run this
##DME VS others

# def define_discriminator2(in_shape=(256,256,1), n_classes=2):
#   in_image = tf.keras.layers.Input(shape=in_shape)

#   fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(in_image)
#   fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
#   fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

#   fe = tf.keras.layers.Dropout(0.2)(fe)

#   fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
#   fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
#   fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

#   fe = tf.keras.layers.Dropout(0.2)(fe)

#   fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
#   fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
#   fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

#   fe = tf.keras.layers.Dropout(0.2)(fe)

#   fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
#   fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
#   fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

#   fe = tf.keras.layers.Flatten()(fe)
#   fe = tf.keras.layers.Dropout(0.4)(fe)
#   fe = tf.keras.layers.Dense(n_classes)(fe)

#   c2_out_layer = tf.keras.layers.Activation("softmax")(fe)
#   c2_model     = tf.keras.models.Model(in_image, c2_out_layer)
#   c2_model.compile(loss = "categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00005,  beta_1 = 0.5), metrics = ["accuracy"])
#   #tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2)

#   d2_out_layer = tf.keras.layers.Lambda(custom_activation)(fe)
#   d2_model     = tf.keras.models.Model(in_image, d2_out_layer)
#   d2_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00005, beta_1 = 0.5))
#   return d2_model, c2_model